<h1> SENTIMENT CLASSIFCATION USING VADER, NAIVE BAYES & LOGISTIC REGRESSION

In [259]:
import re
import os
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pymongo
import json

# Connect to Mongo and Get Databases

mongo_connect_string = 'mongodb://TwitterIPO?authSource=TwitterIPO'

client = pymongo.MongoClient(mongo_connect_string)
db = client.TwitterIPO
RawTweets = db.RawTweets
ProcessedTweets = db.ProcessedTweets
PriceData = db.PriceData
Vader = db.Vader
MNB_Logit_SVM_Sentiment = db.MNB_Logit_SVM_Sentiment2

In [260]:
def clean_tweet(t):

    t = t.replace('RT ','')
    t = t.replace('$','')
    t = t.replace('#','')
    
    for word in t:
        if word.startswith(("@")) == True:
            t = t.replace(word,'')
            
    t = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''','',t)
    t = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", t)
    t = t.replace(',','')
    t = t.replace('.','')
    t = t.replace(';','')
    t = t.replace("'",'')
    t = t.replace('"','')
    t = t.replace('...','')
    t = t.lower()
    
    return(t)

In [261]:
# Get all records from RawTweets DB
tweetdb = list(ProcessedTweets.find())

# Clean tweets up. Remove URLs, RTs, @..'s, etc
result = []

for tweet in tweetdb:
    #print(tweet['text'])
    tweet['text'] = clean_tweet(tweet['text'])
    result.append(tweet)

In [262]:
# Create DF with IPO Tweets

df_IPOTWEETS = pd.DataFrame(result)
df_IPOTWEETS = df_IPOTWEETS[df_IPOTWEETS['lang'] == 'en']
df_IPOTWEETS.head()

,UID,_id,company,datems,datepy,lang,text,tweet_id
0,743562292776730624_ACIA,58f187361845122414a0b5ce,ACIA,1466114039000,2016-06-16 13:53:59,en,acia a picture is worth a thousand words marke...,743562292776730624
1,743488689129594881_ACIA,58f187361845122414a0b5cf,ACIA,1466096491000,2016-06-16 09:01:31,en,acia a very powerful hightight flag set up tha...,743488689129594881
2,743489909328089088_ACIA,58f187361845122414a0b5d0,ACIA,1466096782000,2016-06-16 09:06:22,en,perfect on acia ross! 3932 4310\nif you need a...,743489909328089088
3,743590750559027201_ACIA,58f187361845122414a0b5d1,ACIA,1466120824000,2016-06-16 15:47:04,en,acia scwx mrk pen z zg mdvn veev some idea we ...,743590750559027201
4,743485193542049794_ACIA,58f187361845122414a0b5d2,ACIA,1466095658000,2016-06-16 08:47:38,en,acia unusually strong name,743485193542049794


<h3> Incorporate Kaggle/UMich/Senti140 Dataset (1.5m tagged tweets - not all by humans though)

In [278]:
## Use this to incorporate the 1.5m tweet dataset
## Transform Dataset


#df_senti15 = pd.read_csv('Senti_Kaggle_UMICH.csv', error_bad_lines=False)
#df_senti15 = df_senti15[['Sentiment','SentimentText']]
#df_senti15 = df_senti15.rename(columns={'Sentiment' : 'label_num','SentimentText' : 'text'})

df_senti15 = df_senti15[['text','label_num']]
df_senti15['label'] = df_senti15['label_num'].map({0 : 'Negative', 1 : 'Positive'})

print(df_senti15.shape)

(1578613, 3)


C:\Users\Saifi\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<h4> Import NLTK Corpus/Brown/ and Create Training Dataset

In [280]:
# Retrieve NLTK Twitter Data and save to CSVs
from nltk.corpus import twitter_samples

path_list = [twitter_samples.abspath(i) for i in twitter_samples.fileids()]

from nltk.twitter.common import json2csv

for file in path_list:
    with open(file) as fp:
        json2csv(fp, os.getcwd() + "\\" + str(file)[-20:-5] +'.csv',['text'])
        
# Read positive and negative tweets from NLTK CSVs

df_neg = pd.DataFrame.from_csv('negative_tweets.csv' , index_col=None)
df_pos = pd.DataFrame.from_csv('positive_tweets.csv', index_col=None)

df_test = pd.DataFrame.from_csv('20150430-223406.csv', index_col=None)


df_neg['label'] = 'Negative'
df_pos['label'] = 'Positive'

In [281]:
# Join and assign labels to the dataset
df_train = pd.concat([df_neg, df_pos,df_senti15], ignore_index=True)
df_train['label_num'] = df_train.label.map({'Negative':0, 'Positive':1})

In [282]:
# Read Sander's Corpus
df_Brown = pd.read_csv('sanders_corpus.csv')
df_Brown.head()

# Assign column labels
df_Brown = df_Brown[['TweetText','Sentiment']]
df_Brown.columns = ['text','label']

# Remove Irrelevant and Neutral
df_Brown = df_Brown[df_Brown['label'] != 'irrelevant']
df_Brown = df_Brown[df_Brown['label'] != 'neutral']
df_Brown['label_num'] = df_Brown.label.map({'negative' : 0, 'positive' : 1})

# Join Brown Corpus to NLTK Twitter Data
df_train = pd.concat([df_train, df_Brown],ignore_index=True)

In [283]:
## Clean Tweets
df_train['text'] = df_train['text'].map(lambda x: clean_tweet(x))
print(df_train.shape)
df_train['text'][:10]

(1589704, 3)


0                                  hopeless for tmr :(
1    everything in the kids section of ikea is so c...
2    hegelbon that heart sliding into the waste bas...
3    “ketchburning: i hate japanese call him bani :...
4               dang starting next week i have work :(
5                           oh god my babies faces :( 
6                     rileymcdonough make me smile :((
7    f0ggstar stuartthull work neighbour on motors ...
8                            why?:(tahuodyy: sialan:( 
9    athabasca glacier was there in :-( athabasca g...
Name: text, dtype: object

<h1>Accuracy for 75/25 Split of NLTK Twitter Corpus

In [284]:
# Assign X and Y
X = df_train['text']
y = df_train['label_num']

# split X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer(ngram_range=(1, 3))
#vect = CountVectorizer()

# learn training data vocabulary, then use it to create a document-term matrix
X_train_dtm = vect.fit_transform(X_train)
# examine the document-term matrix
X_train_dtm
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

# calculate accuracy of class predictions
from sklearn import metrics
print("Naive Bayes Accuracy", metrics.accuracy_score(y_test, y_pred_class))

# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

# Check False Pos and False Neg (change sign < or >)
#X_test[(y_pred_class > y_test)]

# calculate predicted probabilities for X_test_dtm (poorly calibrated)
#y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
#y_pred_prob

# calculate AUC
#metrics.roc_auc_score(y_test, y_pred_prob)

############# LOGISTIC Regression ###############

# import and instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

# train the model using X_train_dtm
%time logreg.fit(X_train_dtm, y_train)

# make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

# calculate accuracy
print("LOGISTIC Regression Accuracy", metrics.accuracy_score(y_test, y_pred_class))

# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

# print message text for the false positives (ham incorrectly classified as spam)
#X_test[(y_pred_class==1) & (y_test==0)]

# calculate predicted probabilities for X_test_dtm (well calibrated)
#y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
#y_pred_prob

# calculate AUC
#metrics.roc_auc_score(y_test, y_pred_prob)

########## SVM ##############

from sklearn.feature_extraction.text import TfidfVectorizer

tfid_vect = TfidfVectorizer(sublinear_tf=True,
                             use_idf=True,
                           ngram_range=(1,3))
train_vectors = tfid_vect.fit_transform(X_train)
test_vectors = tfid_vect.transform(X_test)

from sklearn import svm

# Train the classifier
classifier_rbf = svm.LinearSVC()
classifier_rbf.fit(train_vectors, y_train)

# Predict the classifier
prediction_rbf = classifier_rbf.predict(test_vectors)

# Add to dataframe
print("SVM Accuracy", metrics.accuracy_score(y_test, prediction_rbf))

(1192278,)
(397426,)
(1192278,)
(397426,)
Wall time: 1.52 s
Naive Bayes Accuracy 0.804099379507
Wall time: 16min 43s
LOGISTIC Regression Accuracy 0.825514687011
SVM Accuracy 0.827391766014


<h2> BUILD DOCUMENT MATRIX FOR IPO TWEETS

In [285]:
# Define X and y (from the Twitter data) for use with COUNTVECTORIZER
X_train = df_train.text
y_train = df_train.label_num

print(X_train.shape)
print(y_train.shape)

# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer(ngram_range=(1, 3))
#vect = CountVectorizer()

# learn training data vocabulary, then use it to create a document-term matrix
X_train_dtm = vect.fit_transform(X_train)
# examine the document-term matrix
X_train_dtm

(1589704,)
(1589704,)


<1589704x15212304 sparse matrix of type '<class 'numpy.int64'>'
	with 52758599 stored elements in Compressed Sparse Row format>

In [286]:
X_test_dtm = vect.transform(df_IPOTWEETS.text)
X_test_dtm

<23749x15212304 sparse matrix of type '<class 'numpy.int64'>'
	with 418465 stored elements in Compressed Sparse Row format>

<h3> Naive Bayes

In [287]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# Naive Bayes Prediction
%time nb.fit(X_train_dtm, y_train)

y_pred_class = nb.predict(X_test_dtm)

df_IPOTWEETS['mnb_predict'] = y_pred_class

Wall time: 2.34 s


<h3> LOGIT Predictor

In [288]:
# import and instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

# train the model using X_train_dtm
%time logreg.fit(X_train_dtm, y_train)

Wall time: 26min 35s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [289]:
y_pred_class = logreg.predict(X_test_dtm)
df_IPOTWEETS['logit_predict'] = y_pred_class

<h3> LinearSVM </h3>

In [290]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfid_vect = TfidfVectorizer(sublinear_tf=True,
                             use_idf=True,
                           ngram_range=(1,3))
train_vectors = tfid_vect.fit_transform(X_train)
test_vectors = tfid_vect.transform(df_IPOTWEETS['text'])

from sklearn import svm

# Train the classifier
classifier_rbf = svm.LinearSVC()
classifier_rbf.fit(train_vectors, y_train)

# Predict the classifier
prediction_rbf = classifier_rbf.predict(test_vectors)

# Add to dataframe
df_IPOTWEETS['svm_predict'] = prediction_rbf

In [291]:
### Add to MongoDB

for index,row in df_IPOTWEETS.iterrows():
    senti_dict = {}
    senti_dict['UID'] = row['UID']
    senti_dict['mnb_predict'] = row['mnb_predict']
    senti_dict['logit_predict'] = row['logit_predict']
    senti_dict['svm_predict'] = row['svm_predict']
    MNB_Logit_SVM_Sentiment.insert_one(senti_dict)


<h2> VADER Scores </h2>

In [ ]:
#### Sentiment Analysis using Vader
#### Test Version

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sentiment = []

sid = SentimentIntensityAnalyzer()
for index,row in df_IPOTWEETS.iterrows():
    #print(sentence)
    ss = sid.polarity_scores(row['text'])
    ss['UID'] = row['UID']
    Vader.insert_one(ss)
    
# Check Vader Collection
print(Vader.count())
#print(Vader.find_one())